In [1]:
%config InlineBackend.figure_formats = ['svg']
import quimb.tensor as qtn
import quimb as qu
import cotengra as ctg
import autoray as ar
import register_ as reg
import algo_cooling as algo
import quf
import time
import numpy as np
from tqdm import tqdm
import nlopt
import torch
from torch.nn.utils import parameters_to_vector, vector_to_parameters
from quimb.tensor.belief_propagation.l2bp import L2BP


import matplotlib as mpl
import matplotlib.pyplot as plt


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7feed1328a90>>
Traceback (most recent call last):
  File "/home/reza.haghshenas@quantinuum.com/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 

KeyboardInterrupt



In [ ]:
reg.reg_complex_svd()

to_backend = algo.backend_torch(device = "cuda", dtype = torch.float64, requires_grad=False)
to_backend_c = algo.backend_torch(device = "cuda", dtype = torch.complex128, requires_grad=False)
to_backend_ = algo.backend_torch(device = "cuda", dtype = torch.float64, requires_grad=False)

opt = algo.opt_(progbar=False, max_time="rate:1e9", max_repeats=128, optlib="cmaes")


In [ ]:
info_su = qu.load_from_disk(f"store_state/info_su")

In [ ]:
#ITF params
J, h, chi, dt, depth = info_su["J"], info_su["h"], info_su["chi"], info_su["dt"], info_su["depth"],
Lx, Ly, L = info_su["Lx"], info_su["Ly"], info_su["L"]
t = depth * dt
print("t", t, "chi", chi)

In [ ]:
edges_1d, sites, site_tags = info_su["edges_1d"], info_su["sites"], info_su["site_tags"]

In [ ]:
pepo, gauges = info_su["pepo"], info_su["gauges"]
map_tags_2d = info_su["map_tags_2d"]


In [ ]:
depth_total = 4
params = {}

dt_guess = 0.05
for depth in range(depth_total):
        params[f"rx_depth{depth}"] = to_backend_( torch.tensor( -h * dt_guess ).clone().detach() )
        params[f"rzz_depth{depth}"] = to_backend_( torch.tensor( -J * 2 * dt_guess ).clone().detach() )


In [ ]:
%%time
chi = 16
cost_opts = { "Lx":Lx, "Ly":Ly, "depth_total":depth_total, "opt":opt, "edges":edges_1d, "stable":False  }
cost_opts |= { "cutoff":0.0, "map_tags_2d":map_tags_2d, "opt":opt, "chi":chi, "sites": sites }
cost_opts |= { "renorm":False, "equalize_norms":False }
cost_opts |= { "pepo":pepo, "gauges":gauges  }

cost = algo.cost_function_su(params ,  **cost_opts)
print( float(cost) )

In [ ]:
params

In [ ]:
step_size = 0.7
import math, torch

n = int((2*math.pi)/step_size)

# torch.linspace has no endpoint argument — so do this:
thetas = torch.linspace(-math.pi, math.pi, n, dtype=torch.float64)

# remove the last point so we don't duplicate -pi and pi
if thetas[-1] == math.pi:
    thetas = thetas[:-1]

strs = list(params.keys())
pbar = tqdm(total=4*len(strs), desc="dmrg", ncols=100, dynamic_ncols=True)

for _ in range(4):
    for str_ in strs:
        
        cost_ = algo.cost_function_su(params ,  **cost_opts)
        
        cost_l = []
        param_l = []
        param_init = float(params[str_])
        for theta in thetas:
            params[str_] = to_backend_(theta)
            cost = algo.cost_function_su(params,  **cost_opts)
            cost_l.append( cost )
            param_l.append( theta )
        
        params[str_] = to_backend_(param_init)
        i_min = int(np.argmin([float(c) for c in cost_l]))
        best_cost  = float(cost_l[i_min])
        best_theta = param_l[i_min]
        if float(best_cost) < float(cost_):
            params[str_] = to_backend_(best_theta)

        pbar.set_postfix({
                "loss": f"{float(cost_):.3e}",
            })
        pbar.update(1)
            

In [ ]:
cost = algo.cost_function_su(params ,  **cost_opts)
print( float(cost) )
params

In [ ]:
step_size = 0.2
import math, torch

n = int((2*math.pi)/step_size)

# torch.linspace has no endpoint argument — so do this:
thetas = torch.linspace(-math.pi, math.pi, n, dtype=torch.float64)

# remove the last point so we don't duplicate -pi and pi
if thetas[-1] == math.pi:
    thetas = thetas[:-1]

strs = list(params.keys())
pbar = tqdm(total=4*len(strs), desc="dmrg", ncols=100, dynamic_ncols=True)

for _ in range(4):
    for str_ in strs:
        
        cost_ = algo.cost_function_su(params ,  **cost_opts)
        
        cost_l = []
        param_l = []
        param_init = float(params[str_])
        for theta in thetas:
            params[str_] = to_backend_(theta)
            cost = algo.cost_function_su(params,  **cost_opts)
            cost_l.append( cost )
            param_l.append( theta )
        
        params[str_] = to_backend_(param_init)
        i_min = int(np.argmin([float(c) for c in cost_l]))
        best_cost  = float(cost_l[i_min])
        best_theta = param_l[i_min]
        if float(best_cost) < float(cost_):
            params[str_] = to_backend_(best_theta)

        pbar.set_postfix({
                "loss": f"{float(cost_):.3e}",
            })
        pbar.update(1)
            